# Setup

In [ ]:
import numpy as np
from scipy.interpolate import interp1d

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from dustpylib.substructures import gaps
from dustpylib.grid.refinement import refine_radial_local
import dustpy
from dustpy import constants as c

year = c.year
au = c.au

$T(r) = 200~\mathsf{K}  \, (r/\mathsf{au})^{-1/2}$

$\Sigma_g(r) = {200}\,\mathsf{g cm}^{-2} (r/\mathsf{au})^{-1} \, \exp\left(-\frac{r}{r_c}\right)$

$M_\star = M_\odot$


## Run Dustpy
Takes of the order of an hour

In [ ]:
add_planets = True

In [ ]:
planets = {}
if add_planets:
    planets['jupiter'] = {
        'a': 20 * c.au,
        'a_bump': 28.713 * au,
        'M': 317.8 * c.M_earth,
    }

Create simulation, add planet(s) and refine grid around them (just like in [dustpylib](https://dustpylib.readthedocs.io/en/latest/planetary_gaps.html#Duffell-(2020)))

In [ ]:
s = dustpy.Simulation()
s.ini.gas.alpha = 1e-3
s.ini.dust.vfrag = 1000.
s.ini.grid.Nr = 200

s.ini.grid.mmin = 2e-13
s.ini.grid.mmax = 20000.0

# we define a refined radial grid
ri = np.geomspace(s.ini.grid.rmin, s.ini.grid.rmax, s.ini.grid.Nr)
for planet in planets.values():
    ri = refine_radial_local(ri, planet['a_bump'], num=5)

# we assign that grid and use it to set Sigma_gas and T_gas before initialization
s.grid.ri = ri
s.makegrids()
Sigma0 = 200 * (c.au / s.grid.r) * np.exp(-s.grid.r / s.ini.gas.SigmaRc) + 1e-100
s.gas.addfield('Sigma', Sigma0, description='Surface density [g/cm²]')
s.gas.addfield('T', 200 * np.sqrt(c.au / s.grid.r), description='Temperature [K]')

# add the planets and initialize

if add_planets:
    s.addgroup('planets', description='Planets')
    for name, planet in planets.items():
        s.planets.addgroup(name, description=f'Planet {name.title()}')
        s.planets.__dict__[name].addfield('M', planet['M'], description='Mass in g')
        s.planets.__dict__[name].addfield('a', planet['a'], description='Semi-major axis in cm')

s.initialize()

After initialization: prevent the large arrays from being stored

In [ ]:
s.dust.v.rel.azi.save = False
s.dust.v.rel.brown.save = False
s.dust.v.rel.rad .save = False
s.dust.v.rel.tot .save = False
s.dust.v.rel.turb.save = False
s.dust.v.rel.vert.save = False
s.dust.kernel.save = False
s.dust.p.frag = False
s.dust.p.stick = False
s.t.snapshots = np.geomspace(1e2, 3e6, 50) * year

s.writer.overwrite = True
s.writer.datadir = f'data_{(not add_planets) * "no"}bump'
s.verbosity = 10

Bump parameters

In [ ]:
alpha0 = s.gas.alpha.copy()

In [ ]:
def alpha(s):
    # Unperturbed profile
    alpha = alpha0.copy()
    
    # Iteration over all planets
    for name, p in (item for item in s.planets.__dict__.items() if not item[0].startswith('_')):
            
        # Dimensionless planet mass
        q = p.M / s.star.M
        
        # Interpolation of aspect ratio and alpha0 onto planet position
        h = interp1d(s.grid.r, s.gas.Hp / s.grid.r)(p.a)
        alp = interp1d(s.grid.r, alpha0)(p.a)
        
        # Inverse alpha-profile
        alpha /= gaps.duffell2020(s.grid.r, p.a, q, h, alp)
        
    return alpha

In [ ]:
if add_planets:
    # set this function as updater of alpha
    s.gas.alpha.updater = alpha
    s.update()
    
    # change initial condition
    s.gas.Sigma[...] /= s.gas.alpha / alpha0
    s.dust.Sigma[...] /= (s.gas.alpha / alpha0)[:, None]

In [ ]:
f, ax = plt.subplots()
ax.set_xlim(s.grid.r[[0, -1]] / au)
ax.set_ylim(1e-3, 1e3)
ax.loglog(s.grid.r / au, s.gas.Sigma, '+')
ax.loglog(s.grid.r / au, s.dust.Sigma.sum(-1), '+')

In [ ]:
s.update()
s.run()